# Analyzing SQL Workloads

This Jupyter Notebook contains examples of how to use the open APIs of the Db2 Data Management Console to run SQL scripts and use Visual Explain in to analyze SQL. You will start with single SQL statements and work up to running multiple statements against multiple databases to simulate a production workload. By measuring performance over several runs you can identify the slowest statement and the slowest database. Finally you can experiment with different indexes to see if you can improve the performance of the workload. 

The Db2 Data Management Console is more than a graphical user interface. It is a set of microservices that you can use to build custom applications to automate your use of Db2.

This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

You can also embed elements of the full user interface into an IFrame by constructing the appropriate URL.

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

### Import Helper Classes
For more information on these classes, see the lab: [Automate Db2 with Open Console Services](http://localhost:8888/notebooks/Db2_Data_Management_Console_Overview.ipynb)

In [ ]:
%run ./dmc_setup.ipynb

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [ ]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
profile  = 'Sample'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
profileURL = "?profile="+profile
databaseAPI = Db2(Console+'/dbapi/v4')

if databaseAPI.authenticate(user, password, profile) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

### Confirm the connection
To confirm that your connection is working you can check the status of the moitoring service. You can also check your console connection to get the details of the specific database connection you are working with. Since your console user id and password may be limited as to which databases they can access you need to provide the connection profile name to drill down on any detailed information for the database.

In [ ]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

## Accessing the Db2 Console through custom URLs
This lab provides direct links to specific pages in the Db2 Console to make it easy to navigate during the lab. Since you can use this lab from a browser running in the virtual desktop or from your own browser on your own desktop, we just need to save the location of the console you are using. 

By default it is setup to use links that will work from the virtual machine desktop. To run from your own browser on your own desktop, enter the port location of the Db2 Console provided in your welcome note and run the cell below.

In [ ]:
externalConsole = "http://services-uscentral.skytap.com:18638"
database = externalConsole

Testing the Custom URL
Run the cell below.

It will generate a link to the console that works for your selected environment. Notice that it brings up the whole console including the full navigation menu.

Right click on the link and select Open Link in New Tab or Open Link in New Window

In [ ]:
print(database+'/console/?mode=compact#explore/table?profile=Sample')

### Using the Db2 class to run SQL
You can use the console API to run single SQL statement or sets of statements in a single call. A step by step explanation of running SQL through the Db2 Data Management Console SQL service is available in the [Db2 SQL with RESTful Services](http://localhost:8888/notebooks/Db2_RESTful_APIS.ipynb) Jupyter notebook. These examples use the Db2 class defined in [dmc_setup.ipynb](http://localhost:8888/notebooks/dmc_setup.ipynb). The Db2 class takes care of creating a reusable authentication key from your login information. It includes two routines **runSQL** and **getSQLJobResult** that make submitting SQL scripts easy. **runSQL** uses the console service to run one or more statements in the background, saving the result. **getSQLJobResult** lets you access the results of each statement.  

In this first example we will run a single statement. The first step defines a single statement, runs the SQL and retrieves a unique id to identify the SQL running in the background. Using that id you can check the job results to see if it is still running or complete. 

In [ ]:
#Run SQL Statement and Retrieve the identifier used to track the run

# Define the SQL to Run
sqlText = 'select TABSCHEMA, TABNAME from syscat.tables'

# Run SQL throught the Console SQL Editor REST Service
r = databaseAPI.runSQL(sqlText)

# Retrieve the run identifier from the SQL Editor Service
runID = databaseAPI.getJSON(r)['id'] 

# Using the saved runID retrieve a JSON description of the run
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID)) 

# Extract the status of the run from the JSON return and print the runID as well as its status
if json['results'] == [] :
    print(runID+" "+json['status']) 
else :
    print('JSON Results')
    print(json['results']) 

In the next step we check that the statement has finished running. Then we check for three possible conditions. First, if there was an error in the SQL Statement. If so, we print the error message. Second, did the statement run successfully and is there a result set. If so, we print out the results as a table. And third, there are no errors but also no results are returned. Try running different SQL Statements to see the different results. 

In [ ]:
# ONLY RUN THIS STEP IF THERE WERE NO RESULTS FROM THE PREVIOUS STEP
# Even if the status is running. You need to extract and process the results before you get more results.
# Once you have retrieved the results you cannot run the same call a second time successfully.
# The JSON structure is only available on the first call.
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
print(json)

Now let us unpack the JSON that returned in the cell above. The code below checks that the statement completed and checks for errors. If there are no errors and rows were returned in a result set, then the rows are extracted to a dataframe and displayed. Dataframes are a very powerful part of the Python Panda's library. They let us easily manupulate data sets that come back from Db2. 

In [ ]:
if json['results'] != []:
        results = json['results'][0]
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. No results')

You do not want to have to continously check to see if a statement is complete. So let us use a simple loop to do the checking for us. 

In [ ]:
# Run SQL Statement and Retrieve the identifier used to track the run
sqlText = 'select TABSCHEMA, TABNAME from syscat.tables'
runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 
print('Run Identifier: '+str(runID))

# Check to see if the statement finished running
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

# If the statement still has not finished wait in one second intervals
# we check for results because depending on timing it is possible to retrieve results without the status being complete.
# Once we retrieve the results there are removed from the service. 
while json['results'] == [] :
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
    time.sleep(1) 
    
# Assuming we only have one statement, unpack the results    
results = json['results'][0]
if 'error' in results : 
    print(results['error'])
elif 'rows' in results :
    df = pd.DataFrame(results['rows'],columns=results['columns'])
    print(df)
else :
    print('No errors. No results')

### SQL Scripts Used in the Lab
We are going to define a few scripts that we will use during this lab. The first two will define tables that are used during the rest of the lab. The third defines a workload we will reuse. Notice that these scripts contain multiple SQL statements. We are going to have to update our code to handle that.

In [ ]:
sqlScriptCreateEmployee = \
'''
CREATE TABLE EMPLOYEES (ENO INTEGER, DEPTNO INTEGER, LASTNAME VARCHAR(30),
    HIREDATA DATE, SALARY INTEGER);

INSERT INTO EMPLOYEES
-- generate 500000 records
    WITH DT(ENO) AS (VALUES(1) UNION ALL SELECT ENO+1 FROM DT WHERE ENO < 500000)

-- Now, use the generated records in DT to create other columns
-- of the employee record.
    SELECT ENO,
    RAND() * 500,
    TRANSLATE(CHAR(INTEGER(RAND()+500000)),
    CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
        WHEN 1 THEN 'aeiou' || 'hjklm'
        WHEN 2 THEN 'aeiou' || 'npqrs'
        WHEN 3 THEN 'fredr' || 'annab'
        ELSE 'aeiou' || 'twxyz' END,
        '1234567890') AS LASTNAME,
   CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
   INTEGER(1000*RAND()*200) AS SALARY
   FROM DT;

SELECT COUNT FROM EMPLOYEES;

SELECT * FROM EMPLOYEES ORDER BY ENO;
'''
print("Defined Create Employee Script")

In [ ]:
sqlScriptCreateDepartment = \
'''
CREATE TABLE DEPARTMENTS (DEPTNO INTEGER, DEPTNAME VARCHAR(30), REVENUE BIGINT);

INSERT INTO DEPARTMENTS
-- generate 500 department records
WITH DT(DEPTNO) AS (VALUES(1) UNION ALL SELECT DEPTNO+1 FROM DT WHERE DEPTNO < 500 )

    SELECT DEPTNO,
        TRANSLATE(CHAR(INTEGER(RAND()+100000)),
            CASE MOD(DEPTNO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
                WHEN 1 THEN 'aeiou' || 'hjklm'
                WHEN 2 THEN 'aeiou' || 'npqrs'
                WHEN 3 THEN 'fredr' || 'annab'
                ELSE 'aeiou' || 'twxyz' END,
                '1234567890') AS DEPTNAME,
        BIGINT(100000*RAND()*50000) AS REVENUE
        FROM DT;

SELECT COUNT FROM DEPARTMENTS;

SELECT * FROM DEPARTMENTS;
'''
print("Defined Create Department Script")

In [ ]:
sqlScriptWorkload1 = \
'''
WITH SALARYBY (DEPTNO, TOTAL) AS
    (SELECT DEPT.DEPTNO DNO, SUM(BIGINT(EMP.SALARY)) TOTAL_SALARY
        FROM EMPLOYEES EMP, DEPARTMENTS DEPT
        WHERE DEPT.DEPTNO = EMP.DEPTNO AND EMP.SALARY > 190000
        GROUP BY DEPT.DEPTNO
        ORDER BY DNO)
SELECT DEPT.DEPTNAME NAME, SALARYBY.TOTAL COST, DEPT.REVENUE, DEPT.REVENUE-SALARYBY.TOTAL PROFIT
FROM SALARYBY, DEPARTMENTS DEPT
WHERE DEPT.DEPTNO = SALARYBY.DEPTNO
AND REVENUE > TOTAL
ORDER BY PROFIT
'''
print("Defined Workload 1 Script")

In [ ]:
sqlScriptWorkload2 = \
'''
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO 
      AND EMP.SALARY < 50000
      AND YEAR(EMP.HIREDATA) > 2010
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO 
      AND EMP.SALARY < 190000
      AND YEAR(EMP.HIREDATA) > 2010
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO, DEPT.REVENUE
  FROM DEPARTMENTS DEPT WHERE DEPT.REVENUE > 450000000;
'''
print("Defined Workload 2 Script")

In [ ]:
addMoreRows = \
'''
INSERT INTO EMPLOYEES
-- generate 500000 records
    WITH DT(ENO) AS (VALUES(1) UNION ALL SELECT ENO+1 FROM DT WHERE ENO < 500000)

-- Now, use the generated records in DT to create other columns
-- of the employee record.
    SELECT ENO,
    RAND() * 500,
    TRANSLATE(CHAR(INTEGER(RAND()+500000)),
    CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
        WHEN 1 THEN 'aeiou' || 'hjklm'
        WHEN 2 THEN 'aeiou' || 'npqrs'
        WHEN 3 THEN 'fredr' || 'annab'
        ELSE 'aeiou' || 'twxyz' END,
        '1234567890') AS LASTNAME,
   CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
   INTEGER(1000*RAND()*200) AS SALARY
   FROM DT;
'''
print("Defined Add More Rows Script")

### Running Multiple SQL Statements in a Single Script
In the following example we run multiple statements in a single API call. When the statement is started we get a run handle we can use to access the results of the SQL execution. Running the script works the same as running a single statement. However, retrieving results is more complex. Each time you check for results you may get the results of one or more statements. Like before, you can only retrieve the results for each statement once. 

In [ ]:
# Run SQL Statement and Retrieve the identifier used to track the run

sqlText = sqlScriptCreateEmployee
runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 
print(runID) 

In [ ]:
# We need to create a loop to check for results 
# Each time we retrieve new results they are added to the fulljson result set

json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
print("Check for Results")
fulljson = json

while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
    print("Check for results")
    for results in json['results'] :
        fulljson['results'].append(results)
    time.sleep(1) 
print('Done')

In [ ]:
# We need to unpack the entire JSON result set
# and display if each statement ran successfully and how long it took
# we also extract how many rows were affected for insert statements

for results in fulljson['results']:
    print('Statement: '+str(results['index'])+': '+results['command'])
    print('Runtime ms: '+str(results['runtime_seconds']*1000))
    if 'error' in results : 
        print(results['error'])
    elif 'rows' in results :
        df = pd.DataFrame(results['rows'],columns=results['columns'])
        print(df)
    else :
        print('No errors. Row Affected: '+str(results['rows_affected']))
    print()
    print('* * * * * * * * * * * * * * *')

### Creating a Routine to Run an SQL Script
To make things easier we can create reusable routines that will included everything we have developed so far. By running the next two steps, you create two routines that you can call by passing parameters to them. 

While we could create a single routine to run SQL and then display the results, we are creating two different routines so that we can display the results differently later in the lab. 

In [ ]:
def runSQL(profile,user, password, sqlText):
    
    if databaseAPI.authenticate(user, password, profile) :

        # Run the SQL Script and return the runID for later reference 
        runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

        # See if there are any results yet for this job
        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        
        # If the REST call returns an error return the json with the error to the calling routine
        if 'errors' in json :
            return json
        # Append the results from each statement in the script to the overall combined JSON result set
        fulljson = json

        while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
            json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

            # Get the results from each statement as they return and append the results to the full JSON 
            for results in json['results'] :
                fulljson['results'].append(results)
            # Wait 250 ms for more results
            time.sleep(0.25) 
        return fulljson
    else :
        print('Could not authenticate')
print('runSQL routine defined')

In [ ]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        print('Runtime ms: '+str(results['runtime_seconds']*1000))
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()
print('displayResults routine defined')

Now call the new routine using the parameters to define the connection profile to use, the userid and password and the SQL Text. In this example we will drop the table we just created. Try running it a second time. You can see that it correctly returns an error message. 

In [ ]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
sqlText = 'DROP TABLE EMPLOYEES'
displayResults(runSQL(profile, user, password, sqlText))

### Running Multiple Scripts across Multiple Databases
Now we will use our new routines to create the Employees and the Department tables across two databases. In this lab we can use the SAMPLE and the HISTORY database. The example below loops through two databases and two scripts to create the required tables.

In [ ]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptCreateEmployee, sqlScriptCreateDepartment]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print('Running Scripts against profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

### Repeating a script across multiple databases
**Step 1:** If we insert too many rows in one step to the EMPLOYEES table we will exceed the capacity of the transaction log, as it is currently configured. So let's add 1 million more rows to the employee table in both HISTORY and SAMPLE by adding 500,000 2 times. Run the cell below.

**Step 2:** To make things more interesting we will measure the difference in performance between the HISTORY and SAMPLE databases. We can make the job harder for one database by adding more rows just to one database. Delete the HISTORY database from the profile list in the cell below. Then change the number of profileReps from 2 to 4 and run the cell again. 


In [ ]:
profileList = ['SAMPLE']
scriptList = [addMoreRows]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 4

for profile in profileList :
    for x in range(0, profileReps):
        print('Repetition number: '+str(x))
        print('Running Scripts against profile: '+profile)
        for script in scriptList :
            json = runSQL(profile, user, password, script)
print('done')

Now we can check the number of rows in the employees table in the SAMPLE and the HISTORY database. There should be 3,500,000 rows in SAMPLE and 1,500,000 in HISTORY

In [ ]:
profileList = ['SAMPLE','HISTORY']
scriptList = ['select count(*) from employees']
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Running Scripts against profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

### Running multiple scripts across multiple databases - Summarized Results
Now that we have our tables created on both databases, we can run workloads and measure their performance. By repeatedly running the scripts across multiple databases in a single Db2 instance we can simulate a real database environemt. 

Instead of using the displayResults routine we are going to capture runtime metrics for each run of the SQL Query workloads so that we can analyze performance. The appendResults routine builds this dataframe with each pass.

runScripts lets use define the database connection profiles we want to run against, the scripts to run, and now many times to repeat the runs for each profile and for each script.

In [ ]:
# This routine builds up a Data Frame containing the run results as we run workloads across databases
def appendResults(df, profile, json) :
    
    error = ''
    rows = 0
    if 'error' in json :
        print('SQL Service Failed')
    else :
        for results in json['results']:
            if 'error' in results : 
                error = results['error']
            if 'rows_affected' in results : 
                rows = results['rows_affected']
            df = df.append({'profile':profile,'index':results['index'], 'statement':results['command'], 'error':error, 'rows_affected': rows, 'runtime_ms':(results['runtime_seconds']*1000)}, ignore_index=True)
        return df
print('appendResults routine defined')

In [ ]:
# This routine runs multistatment scripts across multiple databases. 
# The scripts are run repeatedly for each profile (database)
def runScripts(profileList, scriptList, user, password, profileReps, scriptReps) :

    df = pd.DataFrame(columns=['profile', 'index', 'statement', 'error', 'rows_affected', 'runtime_ms'])
    
    for x in range(0, profileReps):
        print("Running repetition: "+str(x))
        for profile in profileList :
            print("  Running scripts against: "+profile)
            for y in range(0, scriptReps) :
                print("    Running script repetition: "+str(y))
                for script in scriptList :
                    json = runSQL(profile, user, password, script)
                    while 'errors' in json:
                        print('    * Trying again *')
                        json = runSQL(profile, user, password, script)
                    df = appendResults(df, profile, json)
                        
    return df
print('runScripts routine defined')

In [ ]:
# Retrieve the current package cache list 
# Show the first ten as sorted by the statement execution time
def getCurrentPackageCacheListDF(profile) :
    databaseAPI.authenticate(user, password, profile)
    r = databaseAPI.getCurrentPackageCacheList("false")
    if (databaseAPI.getStatusCode(r)==200):
        json = databaseAPI.getJSON(r)
        if json['count'] > 0:  
            df = pd.DataFrame(json_normalize(json['resources']))
            df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
            return df
        else: 
            print('No data returned') 
            return pd.DataFrame()
    else:
        print(databaseAPI.getStatusCode(r))

The next cell loops through a list of databases as well as a list of scripts and run then repeatedly. You an set the number of times the scripts are run against each database and the number of times the runs against both databases is repeated.

In [ ]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptWorkload1, sqlScriptWorkload2]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 2
scriptReps = 4

df = runScripts(profileList, scriptList, user, password, profileReps, scriptReps)

display(df)

Now lets get the contents of the package cache from both databases. You can compare the package cache measurement to your actual runtimes. If you wait too long between your run and when you check the package cache your statements may have already been pushed out of the package cache. You can always try the test run in the cell above again.

In [ ]:
for profile in profileList:
    print('Profile: '+profile)
    packageCacheDF = getCurrentPackageCacheListDF(profile)
    if packageCacheDF.empty == True :
        print("No statements in the package cache")
    else :
        display(packageCacheDF[['stmt_text','stmt_exec_time_ms','stmtid']])

### Analyze Results
Now we can use the results in the dataframe to look at the results statistically. First we can see the average runtime for each statement across the databases.

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtMean = df.groupby(['statement']).mean()
print(stmtMean)

We can also display the total runtime for each statement across databases.

In [ ]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

We can even graph the total run time for all the statements can compare database performance. Since there are more rows in the employees table in the SAMPLE database it takes longer for the queries to run.

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
profileSum = df.groupby(['profile']).sum()
profileSum.plot(kind='bar')
plt.show()

You can also use the Db2 Data Management Console directly to look at the package cache and the package cache history.

Run the following cell and log in using **DB2INST1** as the userid and password. The cell calls the console as a micro-service and includes part of the live user interface into a IFrame directly in this notebook.

In [ ]:
print(database+'/console/?mode=compact#monitor/package_cache'+profileURL)

Change the timeframe from **Latest** to **Last 1 hour** so you can see a history of package cache monitoring.

## Analyze a Single Statement
You can identify the slowest SQL statement and the slowest database to focus your attention. The next two cells will to that by scanning through our previous results.

In [ ]:
print('Slowest SQL Statement')
slowestSQL = stmtSum['runtime_ms'].idxmax()
print(slowestSQL)

In [ ]:
print('Slowest Database')
slowestProfile = profileSum['runtime_ms'].idxmax()
print(slowestProfile)

A very powerful tool that you can include directly into your notebook is Visual Explain. Just like being able to explore the package cache history you can call the micro-service that provides a live intertive interface. In this example we will take the slowest statment on the slowest database and explain the statement. 

**3.1.1 Note:** Unfortunately there was a regression in ability to call this function directly through a URL. It is supported in 3.1 and will be addressed in 3.1.2. If you are currently using 3.1.1 please copy the SQL statement above, copy it into the Db2 Console SQL Editor and select the Visual Explain tool.

In [ ]:
# Visually explain the access plan for an SQL Statement
profileURL = "?profile="+slowestProfile
stmt = slowestSQL.replace('\n', ' ').replace('\r', '')
print(database+'/console/?mode=compact#sql/explain/create/'+stmt +profileURL)

### Comparing Results
Now we can create indexes to see if we can improve the overall performance of the workloads. The next cell will create indexes on the SALARY column of the Employees table as well as the DEPTO of both the DEPARTMENTS and EMPLOYEES tables. We can also update statistics to make sure the optimizer has the best results to work with. 

In [ ]:
createIndexes = \
'''
CREATE INDEX ix_salary
ON EMPLOYEES(SALARY);

CREATE INDEX ix_empdept
ON EMPLOYEES(DEPTNO);

CREATE INDEX ix_dept
ON DEPARTMENTS(DEPTNO);
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [createIndexes]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Creating Indexes for Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

In [ ]:
updateStats = \
'''
call sysproc.admin_cmd('RUNSTATS ON TABLE db2inst1.employees ON KEY COLUMNS and INDEXES ALL');
call sysproc.admin_cmd('RUNSTATS ON TABLE db2inst1.departments ON KEY COLUMNS and INDEXES ALL');
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [updateStats]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Updating Statistics for Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

Lets see if visual explain predicts any improvements.

**3.1.1 Note:** Unfortunately there was a regression in ability to call this function directly through a URL. It is supported in 3.1 and will be addressed in 3.1.2. If you are currently using 3.1.1 please rerun the visual explain analysis through the Db2 Console.

In [ ]:
# Visually explain the access plan for an SQL Statement
profileURL = "?profile="+slowestProfile
stmt = slowestSQL.replace('\n', ' ').replace('\r', '')
print(database+'/console/?mode=compact#sql/explain/create/'+stmt +profileURL)

Now we will repeat the original workload performance run to see if there is a measureable difference in performance. 

In [ ]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptWorkload1, sqlScriptWorkload2]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 2
scriptReps = 4

df = runScripts(profileList, scriptList, user, password, profileReps, scriptReps)

display(df)

In [ ]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

Compare these results to your original performance without indexes.

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
profileSum = df.groupby(['profile']).sum()
profileSum.plot(kind='bar')
plt.show()

### Clean Up

If you want to run the lab again, or try with different indexes, you can use the cells below to drop either the current indexes or the original tables. 

Experiment with different SQL Statements, and different tables to see what your results look like. You can even try to create new cells that will run different workloads on different databases. 

In [ ]:
dropIndexes = \
'''
DROP INDEX ix_salary;
DROP INDEX ix_empdept;
DROP INDEX ix_dept;
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [dropIndexes]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

In [ ]:
dropTables = \
'''
DROP TABLE EMPLOYEES;
DROP TABLE DEPARTMENTS;
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [dropTables]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

#### Credits: IBM 2019-2021, Peter Kohlmann [kohlmann@ca.ibm.com]